In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math
import geemap
import os

from IPython.display import Image

import folium
from folium import plugins
import importlib

import geopandas as gpd
from PIL import Image

In [2]:
#ee.Authenticate()

In [3]:
ee.Initialize()

*** Earth Engine *** Authenticate calls from this Earth Engine Python client will fail after 2022-05-09: please upgrade. https://developers.google.com/earth-engine/guides/python_install


In [4]:
col_sent = ee.ImageCollection("COPERNICUS/S2_SR")

In [5]:
# Isolate Cook County from the Census collection
counties = ee.FeatureCollection('TIGER/2018/Counties')
cookCounty = counties.filter(ee.Filter.eq("GEOID", '17031'))

# Get all US zip codes, then restrict to those that intersect with Cook County, and then
# filter out those that don't overlap at all but only touch on the boundary
# (e.g. some adjacent zip codes in Illinois and Indiana)
zipCodes = (
    ee.FeatureCollection('TIGER/2010/ZCTA5').filterBounds(cookCounty)
    .filter("GEOID10 != '46311'").filter("GEOID10 != '46324'")
    .filter("GEOID10 != '46321'").filter("GEOID10 != '46327'")
    .filter("GEOID10 != '46320'").filter("GEOID10 != '60047'")
    .filter("GEOID10 != '60035'").filter("GEOID10 != '60110'")
    .filter("GEOID10 != '60118'").filter("GEOID10 != '60143'")
    .filter("GEOID10 != '60191'").filter("GEOID10 != '60106'")
    .filter("GEOID10 != '60523'").filter("GEOID10 != '60441'")
    .filter("GEOID10 != '60448'").filter("GEOID10 != '60449'")
    .filter("GEOID10 != '60491'").filter("GEOID10 != '60417'")
)

# Create a geometry that is the union of all the zip codes in Chicago
zipChiGeom = zipCodes.union(100);
#censusTracts = ee.FeatureCollection("TIGER/2010/Tracts_DP1").filterBounds(cookCounty)

#zipCodeIDs = zipCodes.aggregate_array("GEOID10").getInfo()
#pprint(zipCodeIDs, width=90, compact=True)

In [6]:
surfTemp = ee.ImageCollection('MODIS/061/MOD11A1').filterDate('2018-01-01').filterBounds(zipChiGeom)
LST = surfTemp.select('LST_Day_1km');
LSTVis = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]}

In [11]:
fmap = geemap.Map()
fmap.centerObject(cookCounty, 9)
fmap.addLayer(
    landSurfaceTemperature, landSurfaceTemperatureVis,
    'Land Surface Temperature');
#tempChart = ui.Chart.image.regions(landSurfaceTemperature, zipCodes, ee.Reducer.mean(), scale, 'system:zipCodes', 'system:index')
#lst_urban_point = lst.mean().sample(cookCounty, scale).first().get('LST_Day_1km').getInfo()
fmap.addLayer(zipCodes)
#fmap.addLayer()
#fmap
#tempChart

In [14]:
LST
#print(LST.first().propertyNames())
print(LST.aggregate_array('LST_Day_1km'))

ee.List({
  "functionInvocationValue": {
    "functionName": "AggregateFeatureCollection.array",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.select",
                    "arguments": {
                      "bandSelectors": {
                        "constantValue": [
                          "LST_Day_1km"
                        ]
                      },
                      "input": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      }
                    }
                  }
                }
              }
            },
            "collection": {
              "f

In [16]:
# RUNNING THIS WILL DOWNLOAD A CSV CONTAINING MEAN NDVI BY ZIP CODE IN COOK COUNTY
# TO YOUR PERSONAL DOWNLOADS FOLDER. YOU CAN ALTERNATIVELY JUST COPY THE ndvi_chiZip_dataset.csv THAT IS
# IN MY FOLDER TO YOUR OWN ;).
    
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_stats = os.path.join(out_dir, 'chi_ST_stats_2018_01_01.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
#geemap.zonal_statistics(ndvi_Chi, zipCodes, out_stats, statistics_type='MEAN', scale=1000)
geemap.zonal_statistics(LST, zipCodes, out_stats, statistics_type='MEAN', scale=1000)

ndviVP = {'min': -1, 'max': 1, 'palette': ['000FFF', 'FFFFFF', '00FF00']}

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/matheuboucher/Downloads/chi_ST_stats_2018_01_01.csv
